# Belegungsquote der Frauenhäuser in Deutschland

## Load source data

In [1]:
import pandas as pd
import numpy as np

In [2]:
fpath = "frauenhaus_suche.json"

In [3]:
# load data
raw = pd.read_json(fpath, orient="index")
raw["shelter_id"] = raw.index
raw = raw.reset_index(drop=True)

# clean up geometry columns
raw.loc[raw.geography.notnull(), "geometry"] = raw.loc[raw.geography.notnull(), "geography"]

In [4]:
# get keys
keys = pd.read_csv("data/helpers/free_places_key.csv")

In [5]:
# get all timeseries data
df = pd.DataFrame()

for i, row in raw.iterrows():
    
    temp = pd.DataFrame(row["data"])
    temp["shelter_id"] = row["shelter_id"]
    temp["shelter_name"] = row["title"]
    temp["latitude"] = row["geometry"]["coordinates"][1]
    temp["longitude"] = row["geometry"]["coordinates"][0]
    df = pd.concat([df, temp])

In [6]:
# reformat timeseries
df.timestamp = pd.to_datetime(df.timestamp, dayfirst=True)
df["date"] = df.timestamp.dt.date

In [7]:
# add text keys
df.loc[df.freePlaces=="", "freePlaces"] = np.nan
df.freePlaces = df.freePlaces.astype(float)
df = pd.merge(df, keys, on="freePlaces")

# fill nas
df.loc[df.description.isnull(), "description"] = "k.A."

# add simplified NAs
desc_short = {
    "Aufnahme möglich für Frauen mit 4 oder mehr Kindern":"Aufnahme möglich",
    "Aufnahme möglich für Frauen mit 3 Kindern":"Aufnahme möglich",
    "Aufnahme möglich für Frauen mit 2 Kindern":"Aufnahme möglich",
    "Aufnahme möglich für Frauen ohne Kinder, Aufnahme möglich für Frauen mit 1 Kind":"Aufnahme möglich",
    "Aufnahme möglich für Frauen ohne Kinder":"Aufnahme möglich",
    "Keine Aufnahme möglich":"Keine Aufnahme möglich",
    "Aufnahme möglich ohne detaillierte Angabe":"Aufnahme möglich",
    "k.A.":"keine Angabe"
}

df["status"] = df.description.map(desc_short)

In [8]:
# add bundesland
geocoded = pd.read_csv("data/helpers/shelters_geocoded.csv").drop(["latitude","longitude"], axis=1)
df = pd.merge(df, geocoded, on=["shelter_name","shelter_id"])

In [9]:
# load metadata
metadata = pd.read_csv("data/helpers/shelters_metadata.csv")
metadata = metadata.drop(['title'], axis=1)

In [10]:
# get monthyear as column
df["monthyear"] = df.timestamp.dt.to_period("M")

In [11]:
# trim dates
df = df.loc[(df.monthyear >= "2022-01") & (df.monthyear <= "2022-12"),]

In [12]:
# get counts of timestamp to use as denominator
counts = df[["monthyear","timestamp"]].drop_duplicates().groupby(["monthyear"]).count().reset_index()
counts = counts.rename(columns={"timestamp":"timestamp_count"})

In [13]:
# and total counts
n = counts.timestamp_count.sum()

In [14]:
# add data completeness and get shelters with data less than 80% of time
data_completeness = df.groupby(["shelter_id"]).timestamp.count().reset_index()
data_completeness["pct_data_availability"] = data_completeness["timestamp"]/n
to_drop = data_completeness.loc[data_completeness.pct_data_availability <= .8, "shelter_id"].values

In [15]:
# calculate overall number of shelters
shelter_counts = df[["shelter_id","bundesland"]].drop_duplicates().groupby(["bundesland"]).agg(
    n_shelters = ("bundesland", "count")
).reset_index()
shelter_counts

,bundesland,n_shelters
0,Baden-Württemberg,49
1,Bayern,42
2,Berlin,12
3,Brandenburg,10
4,Bremen,3
5,Hamburg,5
6,Hessen,27
7,Mecklenburg-Vorpommern,9
8,Niedersachsen,27
9,Nordrhein-Westfalen,62


In [16]:
# add kreis column
df["kreis"] = df.apply(lambda row : f"{row.bez} {row.gen}", axis=1)

## Overall shelter-level summary

In [17]:
# get counts of status by shelter
df_shelter = df.groupby(["shelter_name","shelter_id","bundesland","kreis","status"]).agg(
    n = ("status", "count")
).reset_index()
df_shelter.head()

,shelter_name,shelter_id,bundesland,kreis,status,n
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt Hamburg,Aufnahme möglich,18
1,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt Hamburg,keine Angabe,1052
2,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,Aufnahme möglich,842
3,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,Keine Aufnahme möglich,228
4,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt Köln,Aufnahme möglich,14


In [18]:
# pivot to wide and fill values
df_shelter_wide = pd.pivot(
    df_shelter,
    index=["shelter_name","shelter_id","bundesland","kreis"],
    columns="status",
    values="n"
).reset_index().replace(np.nan, 0)
df_shelter_wide.head()

status,shelter_name,shelter_id,bundesland,kreis,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt Hamburg,18.0,0.0,1052.0
1,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,842.0,228.0,0.0
2,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt Köln,14.0,1056.0,0.0
3,2. Autonomes Frauenhaus Berlin,2274,Berlin,Kreisfreie Stadt Berlin,0.0,0.0,1070.0
4,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt Köln,19.0,1051.0,0.0


In [19]:
# get columns as pct
df_shelter_wide.iloc[:,-3:] = df_shelter_wide.iloc[:,-3:]/n

# merge with metadata
df_shelter_wide = pd.merge(df_shelter_wide, metadata, on='shelter_id')

df_shelter_wide.head()

,shelter_name,shelter_id,bundesland,kreis,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe,einrichtungsart,Gehbehinderung,Hörbehinderung/Taubheit,Sehbehinderung/Blindheit,Suchtmittelabhängigkeit
0,1. &amp 3. Hamburger Frauenhaus,2382,Hamburg,Kreisfreie Stadt Hamburg,0.016822,0.000000,0.983178,Frauenhaus,True,False,False,False
1,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,0.786916,0.213084,0.000000,Frauenhaus,False,False,False,False
2,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt Köln,0.013084,0.986916,0.000000,Frauenhaus,True,False,False,False
3,2. Autonomes Frauenhaus Berlin,2274,Berlin,Kreisfreie Stadt Berlin,0.000000,0.000000,1.000000,Frauenhaus,False,False,False,False
4,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt Köln,0.017757,0.982243,0.000000,Frauenhaus,False,False,False,False


In [20]:
incomplete = df_shelter_wide.loc[df_shelter_wide['keine Angabe'] >= .20, "shelter_id"].values
not_frauenhaus = df_shelter_wide.loc[df_shelter_wide.einrichtungsart!="Frauenhaus", "shelter_id"].values
to_drop = np.concatenate([incomplete, to_drop, not_frauenhaus])

In [21]:
df_shelter_wide = df_shelter_wide.loc[~df_shelter_wide.shelter_id.isin(to_drop),]

In [22]:
# filter out shelters data and save to csv
df_shelter_wide.to_csv("./data/cleaned/data_by_shelter.csv", index=False)

## Shelter-level data by month

In [23]:
# get count of values per day
df_monthyear = df.groupby(["monthyear","shelter_name","shelter_id","bundesland","kreis","status"]).agg(
    status_count = ("status", "count")
).reset_index()
df_monthyear = df_monthyear.loc[~df_monthyear.shelter_id.isin(to_drop),] # filter out incomplete, dont need to share with lokalen
df_monthyear.head()

,monthyear,shelter_name,shelter_id,bundesland,kreis,status,status_count
1,2022-01,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,Aufnahme möglich,1
2,2022-01,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,Keine Aufnahme möglich,91
3,2022-01,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt Köln,Aufnahme möglich,8
4,2022-01,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt Köln,Keine Aufnahme möglich,84
6,2022-01,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt Köln,Aufnahme möglich,4


In [24]:
# pivot to wide and fill values
df_monthyear_wide = pd.pivot(
    df_monthyear,
    index=["shelter_name","shelter_id","bundesland","kreis","monthyear"],
    columns="status",
    values="status_count"
).reset_index().replace(np.nan, 0)
df_monthyear_wide.head()

status,shelter_name,shelter_id,bundesland,kreis,monthyear,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe
0,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,2022-01,1.0,91.0,0.0
1,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,2022-02,80.0,0.0,0.0
2,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,2022-03,91.0,0.0,0.0
3,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,2022-04,85.0,0.0,0.0
4,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,2022-05,90.0,0.0,0.0


In [25]:
# add total count of timestamp for percent denominator
df_monthyear_wide = pd.merge(df_monthyear_wide, counts, how="outer")
df_monthyear_wide.head()

,shelter_name,shelter_id,bundesland,kreis,monthyear,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe,timestamp_count
0,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,2022-01,1.0,91.0,0.0,92
1,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt Köln,2022-01,8.0,84.0,0.0,92
2,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt Köln,2022-01,4.0,88.0,0.0,92
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,Sachsen,Kreisfreie Stadt Leipzig,2022-01,92.0,0.0,0.0,92
4,AWO Frauenhaus &quotLotte Lemke&quot,2103,Hessen,Landkreis Hochtaunuskreis,2022-01,8.0,84.0,0.0,92


In [26]:
# calculate percentages
df_monthyear_wide.iloc[:,-4:-1] = df_monthyear_wide.iloc[:,-4:-1].apply(lambda x : x / df_monthyear_wide.timestamp_count, axis=0)
df_monthyear_wide.head()

,shelter_name,shelter_id,bundesland,kreis,monthyear,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe,timestamp_count
0,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,2022-01,0.010870,0.989130,0.0,92
1,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt Köln,2022-01,0.086957,0.913043,0.0,92
2,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt Köln,2022-01,0.043478,0.956522,0.0,92
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,Sachsen,Kreisfreie Stadt Leipzig,2022-01,1.000000,0.000000,0.0,92
4,AWO Frauenhaus &quotLotte Lemke&quot,2103,Hessen,Landkreis Hochtaunuskreis,2022-01,0.086957,0.913043,0.0,92


In [27]:
df_monthyear_wide = pd.merge(df_monthyear_wide, metadata, on='shelter_id', how="left")

In [28]:
# drop timestamp camp and save file
df_monthyear_wide = df_monthyear_wide.drop(["timestamp_count"], axis=1)

In [29]:
df_monthyear_wide.head()

,shelter_name,shelter_id,bundesland,kreis,monthyear,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe,einrichtungsart,Gehbehinderung,Hörbehinderung/Taubheit,Sehbehinderung/Blindheit,Suchtmittelabhängigkeit
0,1. Autonomes Frauenhaus,2027,Sachsen,Kreisfreie Stadt Leipzig,2022-01,0.010870,0.989130,0.0,Frauenhaus,False,False,False,False
1,1. Frauenhaus Köln,2140,Nordrhein-Westfalen,Kreisfreie Stadt Köln,2022-01,0.086957,0.913043,0.0,Frauenhaus,True,False,False,False
2,2. Autonomes Frauenhaus Köln,2253,Nordrhein-Westfalen,Kreisfreie Stadt Köln,2022-01,0.043478,0.956522,0.0,Frauenhaus,False,False,False,False
3,4. Frauen*- und Kinderschutzhaus Leipzig,2317,Sachsen,Kreisfreie Stadt Leipzig,2022-01,1.000000,0.000000,0.0,Frauenhaus,False,False,False,False
4,AWO Frauenhaus &quotLotte Lemke&quot,2103,Hessen,Landkreis Hochtaunuskreis,2022-01,0.086957,0.913043,0.0,Frauenhaus,False,False,False,False


In [30]:
df_monthyear_wide.to_csv("./data/cleaned/data_by_month_and_shelter.csv", index=False)

## Filter to only Frauenhäuser with less than 20% keine Angabe and 80% or more of the time data

In [31]:
datenlage = df[["bundesland","shelter_id"]].drop_duplicates()
datenlage["in_analysis"] = datenlage.shelter_id.map(lambda x : x not in to_drop)
datenlage = datenlage.groupby(["bundesland"]).agg(
    total_shelters = ("bundesland", "count"),
    in_analysis = ("in_analysis", sum)
).reset_index()

## Bundesland-level summary

In [32]:
# filter out where no data and mean of each status % by bundesland
bundesland = df_monthyear_wide \
    .groupby(["bundesland"]) \
    .mean() \
    .reset_index()

bundesland = bundesland.drop(["shelter_id","Gehbehinderung","Hörbehinderung/Taubheit","Sehbehinderung/Blindheit","Suchtmittelabhängigkeit"], axis=1)
bundesland = pd.merge(bundesland, datenlage, on="bundesland", how="outer")
bundesland.head()

,bundesland,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe,total_shelters,in_analysis
0,Baden-Württemberg,0.123249,0.835650,0.032801,49,21
1,Bayern,0.165268,0.819829,0.014567,42,19
2,Brandenburg,0.305398,0.658260,0.036343,10,4
3,Hessen,0.059729,0.940036,0.000000,27,27
4,Mecklenburg-Vorpommern,0.207887,0.767201,0.023463,9,5


In [33]:
bundesland.to_csv("./data/cleaned/bundesland_overview.csv", index=False)

## Bundesland and month-level summary

In [34]:
# filter out where no data and mean of each status % by bundesland
bundesland_monthyear = df_monthyear_wide \
    .groupby(["bundesland", "monthyear"]) \
    .mean() \
    .reset_index()

bundesland_monthyear = bundesland_monthyear.drop(["shelter_id","Gehbehinderung","Hörbehinderung/Taubheit","Sehbehinderung/Blindheit","Suchtmittelabhängigkeit"], axis=1)
bundesland_monthyear = pd.merge(bundesland_monthyear, datenlage, on="bundesland", how="outer")
bundesland_monthyear.head()

,bundesland,monthyear,Aufnahme möglich,Keine Aufnahme möglich,keine Angabe,total_shelters,in_analysis
0,Baden-Württemberg,2022-01,0.267391,0.711957,0.020652,49,21
1,Baden-Württemberg,2022-02,0.251250,0.748750,0.000000,49,21
2,Baden-Württemberg,2022-03,0.095761,0.836211,0.042386,49,21
3,Baden-Württemberg,2022-04,0.122129,0.814006,0.063866,49,21
4,Baden-Württemberg,2022-05,0.129101,0.826455,0.044444,49,21


In [35]:
bundesland_monthyear.to_csv("./data/cleaned/monthly_bundesland_overview.csv", index=False)

## Weihnachts Beispiel - 12.25.2022 um 8 Uhr

In [36]:
# filter by data and get relevant variables
weihnacht = df.loc[df.timestamp==pd.to_datetime("2022-25-12 08:01:00", dayfirst=True),] # filter by day
weihnacht = weihnacht[["shelter_name","shelter_id","latitude","longitude","timestamp","description","status","gen","bez","bundesland"]]

In [37]:
# add metadata
weihnacht = pd.merge(weihnacht, metadata, on='shelter_id')

In [38]:
weihnacht.to_csv("./data/cleaned/belegungsstatus_25-12-2022_8-01.csv", index=False)